In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Flatten
import pickle

2022-05-29 20:53:44.656526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-29 20:53:44.656547: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
IoT = pd.read_csv('../../data/test_heterogeneous.csv')
IoT = IoT.iloc[0:5000]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['fridge_temp_condition']=encoder.fit_transform(IoT['fridge_temp_condition'])
IoT['door_state']=encoder.fit_transform(IoT['door_state'])
IoT['door_sphone_signal']=encoder.fit_transform(IoT['door_sphone_signal'])
IoT['door_type']=encoder.fit_transform(IoT['door_type'])
IoT['ml_light_status']=encoder.fit_transform(IoT['ml_light_status'])

IoT.head()

,date-time,fridge_temperature,fridge_temp_condition,door_state,door_sphone_signal,door_type,gps_latitude,gps_longitude,modbus_FC1_Read_Input_Register,modbus_FC2_Read_Discrete_Value,modbus_FC3_Read_Holding_Register,modbus_FC4_Read_Coil,ml_motion_status,ml_light_status,thermostat_current_temperature,thermostat_status,weather_temperature,weather_pressure,weather_humidity
0,31-03-19~12:36:52,13.10,0,0,0,2,0.0,10.0,11287,3537,27844,30527,1,1,25.0,0,31.788508,1.035,32.036579
1,31-03-19~12:36:53,8.65,0,0,0,2,0.0,10.0,11287,3537,27844,30527,0,0,25.0,0,41.630997,1.035,30.886165
2,31-03-19~12:36:54,2.00,1,0,0,2,0.0,10.0,11287,3537,27844,30527,0,0,25.0,0,42.256959,1.035,19.755908
3,31-03-19~12:36:55,4.80,1,0,0,2,0.0,10.0,53287,1463,33518,23014,1,1,25.0,0,49.116581,1.035,78.949621
4,31-03-19~12:36:56,10.70,0,0,0,2,0.0,10.0,53287,1463,33518,23014,1,1,25.0,0,24.017085,1.035,40.001059


In [3]:
x = IoT.drop(['door_type', 'date-time'], axis=1)
y = IoT['door_type']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [4]:
model2 = Sequential()
    
model2.add(GRU(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
model2.add(Dropout(0.2))
model2.add(GRU(units = 64, activation='tanh', return_sequences=True))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid')) 
model2.add(Flatten())
    
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

2022-05-29 20:53:47.515046: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 20:53:47.515242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-29 20:53:47.515289: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-29 20:53:47.515331: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-29 20:53:47.515372: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [5]:
model2.save('../h5s/GRU.h5') 
# filename = '../h5s/GRU.h5'
# pickle.dump(model2, open(filename, 'wb'))

In [6]:
model2.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

Epoch 1/5
125/125 [==============================] - 4s 15ms/step - loss: -3.2687 - accuracy: 0.0082 - val_loss: -4.5523 - val_accuracy: 0.2240
Epoch 2/5
125/125 [==============================] - 1s 11ms/step - loss: -4.2246 - accuracy: 0.7170 - val_loss: -4.5748 - val_accuracy: 1.0000
Epoch 3/5
125/125 [==============================] - 1s 11ms/step - loss: -4.2354 - accuracy: 0.9900 - val_loss: -4.5748 - val_accuracy: 1.0000
Epoch 4/5
125/125 [==============================] - 1s 10ms/step - loss: -4.2355 - accuracy: 0.9985 - val_loss: -4.5748 - val_accuracy: 1.0000
Epoch 5/5
125/125 [==============================] - 1s 11ms/step - loss: -4.2355 - accuracy: 0.9998 - val_loss: -4.5748 - val_accuracy: 1.0000


In [7]:
predictions = model2.predict(x_test)
y_pred = [round(x[0]) for x in predictions]
y_pred

32/32 [==============================] - 1s 4ms/step


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [8]:
for i in range(3):
	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

[6.05, 0.0, 0.0, 0.0, 215.2309451, 224.4105539, 19244.0, 63377.0, 22333.0, 22099.0, 1.0, 1.0, 27.28714657, 1.0, 25.05374945, 10.98815628, 12.34547561] => 1 ( expected  0 )
[5.1, 1.0, 0.0, 0.0, 513.73012, 517.0850022, 22360.0, 32303.0, 22925.0, 34861.0, 0.0, 0.0, 31.33943158, 1.0, 26.17855336, 10.11278367, 46.20515663] => 1 ( expected  2 )
[2.0, 0.0, 0.0, 0.0, 1.899008555, 12.35146885, 9129.0, 21953.0, 17838.0, 1099.0, 0.0, 0.0, 28.52997713, 1.0, 28.5800217, 0.861515857, 18.52571727] => 1 ( expected  2 )


In [9]:
score, acc = model2.evaluate(x_test, y_test, batch_size=1)

print('Test score:', score)
print('Test accuracy:', acc)

1000/1000 [==============================] - 3s 2ms/step - loss: -4.5748 - accuracy: 1.0000
Test score: -4.574777603149414
Test accuracy: 1.0


In [10]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
recall = recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
f1Score = f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
print('Test accuracy:', accuracy)
print('Test precision:', precision)
print('Test recall:', recall)
print('Test f1Score:', f1Score)

Test accuracy: 0.094
Test precision: 0.094
Test recall: 1.0
Test f1Score: 0.17184643510054842
